In [1]:
using Pkg
Pkg.activate("..")
Pkg.resolve()
Pkg.instantiate()
using JPEC, Plots
gr() 

  Activating project at `~/Documents/GitHub/JPEC`
  No Changes to `~/Documents/GitHub/JPEC/Project.toml`
  No Changes to `~/Documents/GitHub/JPEC/Manifest.toml`
Precompiling project...
   5274.5 ms  ? JPEC
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Warning: Replacing docs for `JPEC.DconMod.compute_eigenvalues :: Tuple{JPEC.DconMod.MatrixData, Float64}` in module `JPEC.DconMod`
└ @ Base.Docs docs/Docs.jl:243


Plots.GRBackend()

In [ ]:
# 1. Define the input parameters for the equilibrium solver.
#    - eq_filename: The name of the g-file we just created.
#    - eq_type: "efit" for a standard g-file.
#    - jac_type: "boozer" or "hamada" for the output coordinates.
#    - mpsi, mtheta: Resolution of the output grid.
"""
    equil_control = JPEC.Equilibrium.EquilibriumControl(;
    eq_filename="beta_1.00",        # eq_filename
    eq_type="efit",          # eq_type
    jac_type="boozer",        # jac_type
    grid_type="ldp",
    psilow=0.01,             # psilow
    psihigh=0.994)             # psihigh
"""
#equil_config = JPEC.Equilibrium.EquilibriumConfig(equil_control,JPEC.Equilibrium.EquilibriumOutput())
# 2. Run the main equilibrium setup function.
#    This will read the file, solve the direct problem, and return the final object.
println("Starting equilibrium reconstruction...")


#plasma_eq = JPEC.Equilibrium.setup_equilibrium(equil_config)
plasma_eq = JPEC.Equilibrium.setup_equilibrium("./DIIID_example/equil.toml")
println("Equilibrium reconstruction complete.")

Starting equilibrium reconstruction...


┌ Info: Forcing hamada coordinate jacobian exponents: power_*
└ @ JPEC.Equilibrium /Users/bursche/Documents/GitHub/JPEC/src/Equilibrium/EquilibriumTypes.jl:48


Equilibrium file: DIIID_example/g147131.02300_DIIID_KEFIT
--> Processing EFIT g-file: DIIID_example/g147131.02300_DIIID_KEFIT
--> Parsed from header: nw=129, nh=129
--> All main data blocks parsed successfully.
--> Creating 1D profile splines...
--> 1D Spline fitting complete.
--> Creating 2D psi spline...
--> 2D Spline fitting complete.
--- Starting Direct Equilibrium Processing ---
Finding magnetic axis...
  Iter  1: R = 1.760526, Z = 0.025705, |ΔR|=7.05e-02, |ΔZ|=2.57e-02
  Iter  2: R = 1.758881, Z = 0.024420, |ΔR|=1.64e-03, |ΔZ|=1.29e-03
  Iter  3: R = 1.758880, Z = 0.024419, |ΔR|=6.85e-07, |ΔZ|=8.76e-07
  Iter  4: R = 1.758880, Z = 0.024419, |ΔR|=2.58e-13, |ΔZ|=2.96e-13
Magnetic axis found at R=1.7588803190504143, Z=0.024419144244256175.
Finding inboard separatrix crossing...
  Restart attempt 1/6 with initial R = 1.529160
inboard separatrix found at R=1.0872289007986813.
Finding outboard separatrix crossing...
  Restart attempt 1/6 with initial R = 1.954160
outboard separatrix fo

In [ ]:
# --- Final Code to Generate 2D Color Plots of R(ψ,θ) and Z(ψ,θ) ---
println("\n--- Generating data for 2D color plots ---")
# 1. Define the grid in flux coordinates (ψ_norm, θ_new)
# This part is identical to the previous steps.
equil_control = plasma_eq.config.control
psi_norm_grid = collect(range(equil_control.psilow, equil_control.psihigh, length=equil_control.mpsi + 1))
theta_new_grid = collect(range(0.0, 1.0, length=equil_control.mtheta + 1))
# 2. Evaluate the `rzphi` spline to get the R and Z values.
println("Evaluating the 'rzphi' mapping spline...")
fs_grid = JPEC.Spl.bicube_eval(plasma_eq.rzphi, psi_norm_grid, theta_new_grid)
println("Evaluation complete.")
# 3. Transform the spline output to physical (R, Z) coordinates.
# This calculates R_grid[i,j] = R(ψ[i], θ[j]) and Z_grid[i,j] = Z(ψ[i], θ[j])
rfac_sq = fs_grid[:, :, 1]
eta_term = fs_grid[:, :, 2]
theta_new_mesh = ones(length(psi_norm_grid)) * theta_new_grid'
eta_grid = 2.0 * pi .* (theta_new_mesh .+ eta_term)
rfac_grid = sqrt.(max.(0.0, rfac_sq))
R_grid = plasma_eq.ro .+ rfac_grid .* cos.(eta_grid)
Z_grid = plasma_eq.zo .+ rfac_grid .* sin.(eta_grid)
println("Calculated R and Z grids.")
# 4. Create the 2D color plot for R(ψ,θ)
println("--- Plotting heatmap for R(ψ,θ) ---")
p_r_heatmap = heatmap(
    psi_norm_grid,
    theta_new_grid,
    R_grid',  # Note the transpose ' to match axis dimensions
    title="Major Radius R(ψ, θ)",
    xlabel="Normalized Psi (ψₙ)",
    ylabel="Poloidal Angle (θ_new)",
    colorbar_title="R [m]"
)
display(p_r_heatmap)
# 5. Create the 2D color plot for Z(ψ,θ)
println("--- Plotting heatmap for Z(ψ,θ) ---")
p_z_heatmap = heatmap(
    psi_norm_grid,
    theta_new_grid,
    Z_grid',  # Note the transpose ' to match axis dimensions
    title="Vertical Position Z(ψ, θ)",
    xlabel="Normalized Psi (ψₙ)",
    ylabel="Poloidal Angle (θ_new)",
    colorbar_title="Z [m]"
)
display(p_z_heatmap)
println("\n2D color plots for R and Z have been saved.")

In [ ]:
# --- Code to Plot Psi and Theta Contours in R-Z Space ---
# This code block goes at the end of your existing script.

println("\n--- Plotting Psi and Theta Contours in R-Z Space ---")

# 1. Select a number of contours to display for clarity
num_psi_contours = 11   # e.g., from ψ_norm = 0.0 to 1.0 in steps of 0.1
num_theta_contours = 13 # e.g., every 30 degrees

# 2. Initialize the plot
#    aspect_ratio=:equal is crucial for tokamak plots to look physically correct.
p_flux_surfaces = plot(
    title="Flux Coordinate System Contours in (R, Z)",
    xlabel="R [m]",
    ylabel="Z [m]",
    aspect_ratio=:equal,
    legend=:outertopright
)

# 3. Plot contours of constant ψ (flux surfaces) in blue
#    We loop through the ROWS of the R_grid and Z_grid matrices.
psi_indices = round.(Int, range(1, stop=size(R_grid, 1), length=num_psi_contours))

for i in psi_indices
    # Each row corresponds to a single psi value
    # We must add the last point to the start to close the loop for a smooth plot
    R_surface = [R_grid[i, :]; R_grid[i, 1]]
    Z_surface = [Z_grid[i, :]; Z_grid[i, 1]]
    plot!(p_flux_surfaces, R_surface, Z_surface, label="", color=:blue, linewidth=1.5)
end

# 4. Plot contours of constant θ (angle contours) in red
#    We loop through the COLUMNS of the R_grid and Z_grid matrices.
theta_indices = round.(Int, range(1, stop=size(R_grid, 2), length=num_theta_contours))

for j in theta_indices
    # Each column corresponds to a single theta value
    plot!(p_flux_surfaces, R_grid[:, j], Z_grid[:, j], label="", color=:red, linewidth=1)
end

# 5. Add a legend manually (a common trick in Plots.jl)
plot!(p_flux_surfaces, [], [], color=:blue, label="Constant ψ")
plot!(p_flux_surfaces, [], [], color=:red, label="Constant θ")

# 6. Display and save the final plot
display(p_flux_surfaces)
println("Flux surface contour plot saved as 'flux_surfaces_RZ.png'")

In [ ]:
# --- Code to Plot 1D Profiles from the 'sq' Spline ---
# This code block goes at the end of your existing script.

println("\n--- Plotting 1D Profiles from 'sq' Spline ---")

# 1. To get smooth curves, we'll evaluate the spline on a dense grid,
#    not just on its internal knot points.
psi_eval_grid = range(0.0, 1.0, length=200)

# 2. Evaluate the 'sq' spline across this dense grid.
#    Calling spline_eval with a vector input is efficient and returns a matrix
#    where each column corresponds to one of the splined quantities.
println("Evaluating 'sq' spline on a dense grid...")
evaluated_profiles = JPEC.Spl.spline_eval(plasma_eq.sq, collect(psi_eval_grid), 0)
println("Evaluation complete.")

# 3. Extract each profile into its own variable for clarity.
#    Based on the direct_run implementation:
#    Column 1: F = R*Bt (Toroidal Field Function)
#    Column 2: P*μ₀ (Scaled Pressure)
#    Column 3: Toroidal Flux function (related to dV/dψ_pol)
#    Column 4: q (Safety Factor)
F_profile = evaluated_profiles[:, 1]
P_profile = evaluated_profiles[:, 2]
Flux_profile = evaluated_profiles[:, 3]
q_profile = evaluated_profiles[:, 4]

# 4. Create and display the plots

# Plot 1: Safety Factor (q)
p_q = plot(
    psi_eval_grid,
    q_profile,
    title="Safety Factor Profile",
    xlabel="Normalized Psi (ψₙ)",
    ylabel="q",
    legend=false,
    linewidth=2
)
display(p_q)

# Plot 2: Scaled Pressure (P*μ₀)
p_p = plot(
    psi_eval_grid,
    P_profile,
    title="Pressure Profile",
    xlabel="Normalized Psi (ψₙ)",
    ylabel="P * μ₀ [T²]",
    legend=false,
    linewidth=2
)
display(p_p)


# Plot 3: Toroidal Field Function (F)
p_f = plot(
    psi_eval_grid,
    F_profile,
    title="Toroidal Field Function",
    xlabel="Normalized Psi (ψₙ)",
    ylabel="F = R * B_t [T*m]",
    legend=false,
    linewidth=2
)
display(p_f)

# Plot 4: Combined plot for comparison
p_all_profiles = plot(
    title="1D Equilibrium Profiles",
    xlabel="Normalized Psi (ψₙ)",
    legend=:best
)
plot!(p_all_profiles, psi_eval_grid, q_profile, label="q (Safety Factor)", linewidth=2)
plot!(p_all_profiles, psi_eval_grid, P_profile, label="P*μ₀ (Pressure)", linewidth=2)
plot!(p_all_profiles, psi_eval_grid, F_profile, label="F (Toroidal Field Fn.)", linewidth=2)
# We can also plot the 3rd quantity, though it's less commonly viewed.
# plot!(p_all_profiles, psi_eval_grid, Flux_profile, label="Toroidal Flux Fn.", linewidth=2)
display(p_all_profiles)


println("1D profile plots saved successfully.")